# --------------------------- ROAD TRAFFIC SIGN RECOGNITION -----------------------------

## Importing the required libraries

In [16]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

%matplotlib inline

In [17]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.utils import to_categorical

In [18]:
from PIL import Image
import os

## Data Interpretation and fetching the path of the required directory

In [19]:
train_dataset = pd.read_csv("train.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
train_dataset.head()

### The annotation used in the above dataset can be interpreted as follows:
#### Roi.X1 :  The top-left coordinate of the bounding box is represented as Roi.X1
#### Roi.Y1 :  The top-right coordinate of the bounding box is represented as Roi.X2
#### Roi.X2 :  The bottom-left coordinate of the bounding box is represented as Roi.Y1
#### Roi.Y2 :  The bottom-right coordinate of the bounding box is represented as Roi.Y2

#### Path : The path in the dataset points to the respective image in 'Train' folder which contains various classes of the traffic signs as made available by the German Traffic Sign Recognition Benchmark (GTSRB)

### The same interpretation goes with the 'test.csv' dataset

In [ ]:
test_dataset = pd.read_csv("test.csv")

In [ ]:
test_dataset.head()

In [16]:
data = []
labels = []

classes = 43

cur_path = os.getcwd()

## Appending the traffic sign images and their respective labels as present in the 'Train' dataset to the empty arrays created above

In [18]:
for i in range (classes):
    path = os.path.join(cur_path, 'Train', str(i))
    images = os.listdir(path)
    
    for a in images:
        try:
            image = Image.open(path + '\\' + a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

data = np.array(data)
labels = np.array(labels)

In [ ]:
print(data.shape, labels.shape)

### Performing train_test_split ()

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### Now we need to convert array of labelled data (i.e; from 0 to 43-1 classes) to one hot vector

In [22]:
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

# To classify the images into their respective categories, we will build a CNN model.

## Creating the model architecture

In [23]:
# Creating a Sequential() model by passing a list of layer instances as present in the further steps.
model = Sequential()

# We need to implement a convo layer to extract the features of the images.
# Also, here we are utilising the ReLU activation layer to convert all negative values to zero.
# Convolution Layer 1
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))

# After the construction of the above convolution layer. Now we need to apply max pooling
# This is required so as to reduce the spatial volume of the image before feeding the above convo layer to the Fully Connected layer. 
# Pooling Layer 1
model.add(MaxPool2D(pool_size=(2,2)))

# The dropout layer prevents overfitting and removes some of the neurons while the model is being trained. Hence we are required to implement this layer in our CNN model architechture.
# We are choosing the dropout rate as 0.25 because it helps in gradually increasing the accuracy and reducing the loss.
# Dropout layer 1
model.add(Dropout(rate=0.25))

# Convolution Layer 2
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

# Pooling Layer 2
model.add(MaxPool2D(pool_size=(2,2)))

# Dropout Layer 2
model.add(Dropout(rate=0.25))

# The next step is to implement a Flatten layer to transform a two-dimensional matrix of features into a vector that can be fed into a fully connected neural network classifier.
model.add(Flatten())

# After the implementation of the above steps we are required to create a Fully Connected Layer (Dense Layer).
# Here the results of the above 2 convolutional layers are fed through to generate a prediction.
# To this dense layer we need to again add a dropout layer.
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.25))

# As the problem statement is a mutli classification problem we are required to use softmax in the last layer of the CNN.
model.add(Dense(43, activation='softmax'))

# We need to compile the model with Adam Optimizer as it performs well and categorical_crossentropy because our dataset has multiple classes to be classified.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Predicting the above model architechture over the 'Train' dataset and calculating its accuracy.

#### After building the model architecture, we need to train the model using model.fit(). I tried with batch sizes 32 and 64. The model performed better with 64. Hence I chose the batch_size in model.fit() as 64.

In [ ]:
epochs = 15
history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, validation_data=(X_test, y_test))

## Visualizing the obtained training accuracy & the val_accuracy per epoch by plotting a simple graph between them.

In [ ]:
plt.plot(history.history['accuracy'], label = 'Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Val Accuracy')
plt.title('ACCURACY PER EPOCH')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

## Visualizing the obtained training loss & the val_loss per epoch by plotting a simple graph between them.

In [ ]:
plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Val Loss')
plt.title('LOSS PER EPOCH')
plt.xlabel('Epochs')
plt.ylabel('loss')

## Conclusions from the above two plots:


### >> From the above two plots we can observe that the two acc. (acc and val_acc) are increasing & the two losses (both loss and val_loss) are decreasing. So this indicates that the model has been trained in a good way and we can further train this model over the unseen or the test data.

### >> In the 'LOSS PER EPOCH' plot we can observe that the accuracies (both accuracy and val_accuracy) stopped increasing at the 13th epoch. So, we should stop training the model at this point. Else, the model would overfit. This is the reason why I chose the max number of epochs as 15. 

## Now we need to implement our model architechture over the 'Test' data.

In [27]:
from sklearn.metrics import accuracy_score

In [ ]:
y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
    
X_test=np.array(data)
pred = model.predict_classes(X_test)

print(accuracy_score(labels, pred))

## Saving the prediction results of the 'Test' dataset in a heirarchical data file (i.e; hdf5 format)

In [29]:
model.save('model_RTSR.h5')

## The above saved file can be loaded and utilised in the python file 'interface.py'